In [1]:
from keras import backend as K

def fbeta(y_true, y_pred, threshold_shift=0, beta=1):

   # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

   # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

Using TensorFlow backend.


In [2]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd 
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

# Embedding
max_features = 11000
maxlen = 65
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 128
epochs = 5

In [3]:
wine_df = pd.read_csv('winemag_data_first150k.csv')

clean_wine_df = wine_df.drop_duplicates(subset='description', keep='last')

In [4]:
X = clean_wine_df.description
y = pd.get_dummies(clean_wine_df.variety)

X.shape, y.shape

((97821,), (97821, 632))

In [5]:
texts = [descript for descript in X]  # list of text samples

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [6]:
data = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')

labels = np.asarray(y)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=.1, random_state=42)

Shape of data tensor: (97821, 65)
Shape of label tensor: (97821, 632)


In [7]:
print('Build model...')

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(labels.shape[1]))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy', fbeta, precision, recall])

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 65, 128)           1408000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 65, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 61, 64)            41024     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 15, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 11, 64)            20544     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 2, 64)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 70)                37800 

In [1]:
from IPython.display import Image
Image(url='https://media.giphy.com/media/l41YgdgP5Ia4niY4U/giphy.gif')   

In [8]:
tboard = TensorBoard(log_dir='./cnn_lstm')

In [9]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
          validation_split=0.2,
          callbacks=[tboard])

Train on 70430 samples, validate on 17608 samples
Epoch 1/20
70430/70430 [==============================] - 14s - loss: 3.6740 - acc: 0.1628 - fbeta: 1.5647e-09 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 3.3294 - val_acc: 0.2276 - val_fbeta: 1.5675e-09 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/20
70430/70430 [==============================] - 13s - loss: 3.2415 - acc: 0.2403 - fbeta: 0.0052 - precision: 0.1178 - recall: 0.0027 - val_loss: 3.2690 - val_acc: 0.2281 - val_fbeta: 0.0097 - val_precision: 0.3992 - val_recall: 0.0049
Epoch 3/20
70430/70430 [==============================] - 13s - loss: 3.0398 - acc: 0.2753 - fbeta: 0.1056 - precision: 0.7096 - recall: 0.0583 - val_loss: 2.9639 - val_acc: 0.2861 - val_fbeta: 0.1521 - val_precision: 0.8000 - val_recall: 0.0845
Epoch 4/20
70430/70430 [==============================] - 13s - loss: 2.7477 - acc: 0.3374 - fbeta: 0.2273 - precision: 0.8136 - recall: 0.1335 - val_loss: 2.7526 - val_acc: 0.3416 - val_f

In [11]:
score = model.evaluate(x_val, y_val, batch_size=128)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test fbeta:', score[2])
print('Test Precision:', score[3])
print('Test Recall:', score[4])

9344/9783 [===========================>..] - ETA: 0sTest loss: 2.19062770967
Test accuracy: 0.534396401916
Test fbeta: 0.555897237199
Test Precision: 0.718452677074
Test Recall: 0.454052948972


# I don't know how, but yes

In [12]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_split=0.2)

Train on 70430 samples, validate on 17608 samples
Epoch 1/10
70430/70430 [==============================] - 13s - loss: 1.2634 - acc: 0.7118 - fbeta: 0.7233 - precision: 0.8848 - recall: 0.6126 - val_loss: 2.1963 - val_acc: 0.5349 - val_fbeta: 0.5559 - val_precision: 0.7070 - val_recall: 0.4587
Epoch 2/10
70430/70430 [==============================] - 13s - loss: 1.2335 - acc: 0.7193 - fbeta: 0.7307 - precision: 0.8854 - recall: 0.6230 - val_loss: 2.1528 - val_acc: 0.5325 - val_fbeta: 0.5583 - val_precision: 0.7426 - val_recall: 0.4483
Epoch 3/10
70430/70430 [==============================] - 13s - loss: 1.2088 - acc: 0.7274 - fbeta: 0.7379 - precision: 0.8880 - recall: 0.6322 - val_loss: 2.1888 - val_acc: 0.5351 - val_fbeta: 0.5600 - val_precision: 0.7155 - val_recall: 0.4609
Epoch 4/10
70430/70430 [==============================] - 13s - loss: 1.1791 - acc: 0.7335 - fbeta: 0.7438 - precision: 0.8884 - recall: 0.6406 - val_loss: 2.2479 - val_acc: 0.5384 - val_fbeta: 0.5620 - val_preci

In [13]:
score = model.evaluate(x_val, y_val, batch_size=128)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test fbeta:', score[2])
print('Test Precision:', score[3])
print('Test Recall:', score[4])

9344/9783 [===========================>..] - ETA: 0sTest loss: 2.30032086945
Test accuracy: 0.529898803999
Test fbeta: 0.55713296237
Test Precision: 0.697359951169
Test Recall: 0.464274762279


# Rabbit Holes and Recommendations

- Balancing Class weights in Keras.
- Visualizing Embeddings in Tensorflow.
- Wondering why I chose a dataset with 632 classes.
- Loving that I chose a dataset with 632 classes.
- Fine Tune CNN/LSTM model 

___